In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers rouge-score nltk SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ac5013ed65c8d0ae193ec4e110d7d11bca1e06b44cac72d4b361863fa1902756
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff859

In [ ]:
!pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
import transformers
from datasets import load_dataset, load_metric
import nltk
import torch
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
import torch_optimizer as optim
from transformers.optimization import get_linear_schedule_with_warmup

In [ ]:
#buoild the dataset taking only the first 400000 samples
raw_datasets = load_dataset("liweili/c4_200m", split='train', streaming=True)
dataset200000 = {"inputs": [], "outputs" : []}
c = 0
for i in raw_datasets:
  if c >= 50000: #200000
    break
  dataset200000['inputs'].append(i['input'])
  dataset200000['outputs'].append(i['output'])
  c += 1

#define tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
len(dataset200000['inputs'])

50000

In [ ]:
class Dataset_class:
  def __init__(self, inputs):
    self.input_ids = inputs['inputs']['input_ids']
    self.attention_mask = inputs['inputs']['attention_mask']
    self.target_ids = inputs['outputs']['input_ids']
    self.target_attention_mask = inputs['outputs']['attention_mask']

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, item):
    return [torch.tensor(self.input_ids[item], dtype=torch.long),
            torch.tensor(self.attention_mask[item], dtype=torch.long),
            torch.tensor(self.target_ids[item], dtype=torch.long),
            torch.tensor(self.target_attention_mask[item], dtype=torch.long)]

In [ ]:
def preprocess_data(dataset, tokenizer, prefix, max_length):
  train_dataset = {}
  valid_daatset = {}

  #train inputs
  inputs = [prefix + doc for doc in dataset["inputs"][:45000]] #160000
  train_dataset['inputs'] = tokenizer(inputs, max_length = max_length, truncation = True, padding = True)
  #train outputs
  #with tokenizer.as_target_tokenizer():
  train_dataset['outputs'] = tokenizer(dataset["outputs"][:45000], max_length = max_length, truncation = True, padding = True)

  #valid inputs
  inputs = [prefix + doc for doc in dataset["inputs"][45000:50000]]
  valid_daatset['inputs'] = tokenizer(inputs, max_length = max_length, truncation = True, padding = True)
  #valid outputs
  #with tokenizer.as_target_tokenizer():
  valid_daatset['outputs'] = tokenizer(dataset["outputs"][45000:50000], max_length = max_length, truncation = True, padding = True)

  traindataset = Dataset_class(train_dataset)
  train_dataloader = torch.utils.data.DataLoader(traindataset, shuffle=True, batch_size=4)

  validdataset = Dataset_class(valid_daatset)
  valid_dataloader = torch.utils.data.DataLoader(validdataset, shuffle=False, batch_size=4)

  return train_dataloader, valid_dataloader

In [ ]:
train_dataloader, valid_dataloader = preprocess_data(dataset200000, tokenizer, "correcting: ", 1024)

In [ ]:
#download the model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small") #t5-base
model.to('cuda')
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
#set parameters
class Hparams:
  def __init__(self, train_dataloader, tot_epochs=1, learning_rate=5e-5):
    self.tot_epochs = tot_epochs
    self.training_steps_per_epoch = len(train_dataloader)
    self.total_num_training_steps = int(self.training_steps_per_epoch * tot_epochs)
    self.learning_rate = learning_rate
    self.warmup_steps = int(self.total_num_training_steps * 0.1)


params = Hparams(train_dataloader)
optimizer = torch.optim.AdamW(model.parameters(), lr=params.learning_rate)
#optimizer = AdamW(model.parameters(), lr=params.learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=params.warmup_steps, num_training_steps=params.total_num_training_steps) #to adjust the learning rate during training

In [ ]:
def evaluate_model(model, test_set):
  eval_loss = []
  for inputs in tqdm(test_set, desc="eval test"):
    input_ids, attention_mask, target_ids, target_attention_mask = inputs
    input_ids = input_ids.to('cuda')
    attention_mask = attention_mask.to('cuda')
    target_ids = target_ids.to('cuda')
    target_attention_mask = target_attention_mask.to('cuda')

    with torch.no_grad():
      output = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
      batch_loss = output[0]
    eval_loss += [batch_loss.cpu().item()]
  eval_loss = np.mean(eval_loss)

  print("Average loss on evaluation set = ", eval_loss)

In [ ]:
#training
def training(model, train_dataloader, valid_dataloader, params, optimizer, scheduler):
  print("Tot epochs: ", params.tot_epochs)

  for epoch in range(params.tot_epochs):
      print("Start epoch", epoch + 1)

      train_loss=0
      epoch_iterator = tqdm(train_dataloader, desc='Epoch iteration')
      model.train()
      model.zero_grad()
      for _, inputs in enumerate(epoch_iterator):
        input_ids, attention_mask, target_ids, target_attention_mask = inputs
        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        target_ids = target_ids.to('cuda')
        target_attention_mask = target_attention_mask.to('cuda')
        optimizer.zero_grad()

        output = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        batch_loss = output[0]
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        train_loss += batch_loss.item()
        epoch_iterator.set_description('(batch loss=%g)' % batch_loss.item())
      print("Average train loss = ", train_loss/params.training_steps_per_epoch, " [Epoch: ", epoch+1, "]")

      print("Evaluate on Validation set")
      evaluate_model(model, valid_dataloader)

  return model

In [ ]:
print("loss before training")
evaluate_model(model, valid_dataloader)

loss before training


eval test: 100%|██████████| 20000/20000 [33:54<00:00,  9.83it/s]

Average loss on evaluation set =  2.8632560154557227


In [ ]:
model = training(model, train_dataloader, valid_dataloader, params, optimizer, scheduler)

Tot epochs:  1
Start epoch 1


(batch loss=0.0759583): 100%|██████████| 11250/11250 [1:39:57<00:00,  1.88it/s]


Average train loss =  0.10854372736935815  [Epoch:  1 ]
Evaluate on Validation set


eval test: 100%|██████████| 1250/1250 [02:02<00:00, 10.22it/s]

Average loss on evaluation set =  0.08382724877372384


In [ ]:
#save model
path_model = "/content/drive/MyDrive/EAI/Results/t5_gc.pt"
torch.save(model, path_model)

In [ ]:
print("loss after training")
evaluate_model(model, valid_dataloader)

loss after training


eval test: 100%|██████████| 1250/1250 [02:02<00:00, 10.19it/s]

Average loss on evaluation set =  0.08387039063945413
